In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import (
    Adafactor,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    AutoConfig,
    GPTNeoXForCausalLM,
    AutoModelForCausalLM,
)
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from redditqa.dataset import load_reddit_dataset
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
import os
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import (
    Adafactor,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    AutoConfig,
    GPTNeoXForCausalLM,
    AutoModelForCausalLM,
)
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import (
    pipeline,
    TextGenerationPipeline,
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModel,
    AutoModelForSequenceClassification,
    GPTNeoXForCausalLM,
    LlamaForSequenceClassification,
)
from redditqa.dataset import load_reddit_dataset
from transformers.utils.hub import convert_file_size_to_int, get_checkpoint_shard_files
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    set_peft_model_state_dict,
)
from peft.utils import _get_submodules
import peft
import torch
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats import inter_rater as irr

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-10 12:09:44.785970: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 12:09:45.894824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Loading the eval dataset

In [3]:
eval_dataset = load_reddit_dataset("eval")
eval_dataset = eval_dataset.select(range(5))

Loading cached processed dataset at /scratch1/jhoff/reddit_dataset_cached/eval/cache-cba55e4212677d14.arrow
Loading cached shuffled indices for dataset at /scratch1/jhoff/reddit_dataset_cached/eval/cache-71c7c533e04253a7.arrow


In [4]:
questions = eval_dataset['question_title']

#### Run models

In [5]:
model_checkpoints = {
    'baseline': 'meta-llama/Llama-2-7b-chat-hf',
    'checkpoint-2500': '/scratch1/jhoff/checkpoints/finetuned_llama-2-7b-chat-hf/checkpoint-2500_merged',
    'checkpoint-8500': '/scratch1/jhoff/checkpoints/finetuned_llama-2-7b-chat-hf/checkpoint-8500_merged',
}

Load the models and make sure they actually are different

In [6]:
models = {}
for name, checkpoint_path in model_checkpoints.items():
    print(f"Loading {name} from {checkpoint_path}")
    model = AutoModelForCausalLM.from_pretrained(checkpoint_path, torch_dtype=torch.bfloat16)
    model.config.pad_token_id = model.config.eos_token_id
    models[name] = model

Loading baseline from meta-llama/Llama-2-7b-chat-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
models['baseline'].model.layers[0].self_attn.q_proj.weight

Parameter containing:
tensor([[-0.0060, -0.0146, -0.0021,  ...,  0.0042,  0.0018, -0.0035],
        [ 0.0142, -0.0043,  0.0032,  ..., -0.0092, -0.0108,  0.0073],
        [-0.0137,  0.0121,  0.0002,  ...,  0.0061,  0.0181, -0.0030],
        ...,
        [ 0.0018,  0.0093, -0.0006,  ...,  0.0092, -0.0289,  0.0085],
        [ 0.0249,  0.0116,  0.0035,  ..., -0.0322, -0.0165, -0.0111],
        [-0.0136, -0.0067,  0.0016,  ...,  0.0176,  0.0175, -0.0083]],
       dtype=torch.bfloat16, requires_grad=True)

In [30]:
models['checkpoint-2500'].model.layers[0].self_attn.q_proj.weight

Parameter containing:
tensor([[-0.0057, -0.0152, -0.0024,  ...,  0.0041,  0.0019, -0.0035],
        [ 0.0140, -0.0042,  0.0029,  ..., -0.0090, -0.0108,  0.0072],
        [-0.0139,  0.0126,  0.0006,  ...,  0.0059,  0.0182, -0.0029],
        ...,
        [ 0.0016,  0.0093, -0.0006,  ...,  0.0087, -0.0284,  0.0085],
        [ 0.0250,  0.0115,  0.0036,  ..., -0.0322, -0.0165, -0.0109],
        [-0.0134, -0.0066,  0.0022,  ...,  0.0179,  0.0170, -0.0082]],
       dtype=torch.bfloat16, requires_grad=True)

In [34]:
models['checkpoint-8500'].model.layers[0].self_attn.q_proj.weight

Parameter containing:
tensor([[-5.6152e-03, -1.5503e-02, -2.9297e-03,  ...,  3.3875e-03,
          2.0294e-03, -3.2196e-03],
        [ 1.3977e-02, -4.3335e-03,  2.9144e-03,  ..., -8.3618e-03,
         -1.1108e-02,  7.1106e-03],
        [-1.3855e-02,  1.3062e-02,  1.3199e-03,  ...,  6.7749e-03,
          1.7944e-02, -3.1586e-03],
        ...,
        [ 1.5564e-03,  1.0254e-02,  4.6253e-05,  ...,  7.4158e-03,
         -2.8198e-02,  8.9722e-03],
        [ 2.5513e-02,  1.1597e-02,  3.9368e-03,  ..., -3.1250e-02,
         -1.6724e-02, -1.1108e-02],
        [-1.2939e-02, -8.3008e-03,  1.1063e-03,  ...,  1.9897e-02,
          1.6479e-02, -8.9111e-03]], dtype=torch.bfloat16, requires_grad=True)

In [14]:
template = "<|ELIF|> Question: %question\nAnswer: "

def generate_answers(model_path):

    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.config.pad_token_id = model.config.eos_token_id
    model = model.cuda().eval()

    generation_kwargs = {
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "min_length": 32,
        "max_length": 128,
    }

    pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device='cuda:0')

    answers = {}

    for question in questions:
        prompt = template.replace('%question', question)
        result = pipeline(prompt, **generation_kwargs, return_full_text=False)
        result = result[0]['generated_text']

        answers[question] = result

    return answers


model_answers = {}
for model_name, checkpoint in model_checkpoints.items():
    model_answers[model_name] = generate_answers(checkpoint)

Loading checkpoint shards: 100%|██████████| 2/2 [02:40<00:00, 80.27s/it] 
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OSError: /scratch1/jhoff/checkpoints/finetuned_llama-2-7b-chat-hf/checkpoint-2500 does not appear to have a file named config.json. Checkout 'https://huggingface.co//scratch1/jhoff/checkpoints/finetuned_llama-2-7b-chat-hf/checkpoint-2500/None' for available files.